In [1]:
# import sys
# !{sys.executable} -m pip install gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
import gmaps
import json

# Import API key
from config import g_key

In [3]:
data = "02-Homework_06-Python-APIs_Instructions_output_data_cities.csv"
city_df = pd.read_csv(data, index_col=0)
city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City_ID,,,,,,,,,
0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [4]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [5]:
gmaps.configure(api_key=g_key)

In [6]:
locations = city_df[['Lat','Lng']]
weights = city_df['Humidity']
fig = gmaps.figure(center=(0,0),zoom_level=1.5)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
coordinates = [
    (40.71, -74.00),
    (30.26, -97.74),
    (46.87, -96.78),
    (47.60, -122.33),
    (32.71, -117.16)
]

In [8]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [9]:
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [10]:
ideal_df = city_df.loc[(city_df['Cloudiness'] == 0) & (city_df['Wind Speed'] < 10) & (city_df['Max Temp'] > 70) & (city_df['Max Temp'] < 80)]
ideal_df = ideal_df.dropna(how='any')
ideal_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City_ID,,,,,,,,,
70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [11]:
hotels = []
for index, row in ideal_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    try:
        res = requests.get(f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius=5000&type=lodging&key={g_key}')
        hotel = res.json()['results'][0]['name']
        hotels.append(hotel)
        print(f"Found hotel near {row['City']}: {hotel}")
    except:
        print(f"No hotels found near {row['City']}.")
        hotels.append('No hotels nearby')

Found hotel near ponta do sol: Hotel Escarpas do Lago
Found hotel near cidreira: Hotel Castelo
Found hotel near morondava: La Case Bambou
Found hotel near vaini: Dandeli Lake County
Found hotel near nador: Hôtel La Giralda
Found hotel near mogok: MOGOK HILL HOTEL
Found hotel near birjand: تاکسی سرویس آرامش


In [12]:
hotel_df = ideal_df
hotel_df['Hotel Name'] = hotels
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
City_ID,,,,,,,,,,
70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Hotel Escarpas do Lago
88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Castelo
138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,La Case Bambou
139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Lake County
176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel La Giralda
297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,MOGOK HILL HOTEL
444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,تاکسی سرویس آرامش


In [13]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
marker_layer = gmaps.marker_layer(
    locations,
    info_box_content = hotel_info
)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…